# SQL

Компания купила крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных. В ней содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

Таблица books
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Исследование исходных данных

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

### Таблица books.

In [3]:
query = '''SELECT * FROM books LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица authors.

In [4]:
query = '''SELECT * FROM authors LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица publishers

In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица ratings

In [6]:
query = '''SELECT * FROM ratings LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews

In [7]:
query = '''SELECT * FROM reviews LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Количество книг, вышедших после 1 января 2000 года

In [8]:
query = '''
SELECT count(book_id) AS count_books
FROM books
WHERE publication_date  > '2000-01-01';
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,count_books
0,819


После 1 января 2000 года вышло 819 книг.

### Количество обзоров и средняя оценка для каждой книги

In [9]:
query = '''
SELECT b.title,
       b.book_id,
       COUNT(DISTINCT rv.review_id) AS count_review,
       AVG(r.rating) AS avg_rating
       
FROM books AS b
LEFT OUTER JOIN ratings AS r ON b.book_id=r.book_id
LEFT OUTER JOIN reviews AS rv ON b.book_id=rv.book_id

GROUP BY b.title, b.book_id
ORDER BY avg_rating DESC;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,count_review,avg_rating
0,Arrows of the Queen (Heralds of Valdemar #1),86,2,5.00
1,The Walking Dead Book One (The Walking Dead #...,901,2,5.00
2,Light in August,390,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,972,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,136,2,5.00
...,...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,915,3,2.25
996,His Excellency: George Washington,316,2,2.00
997,Drowning Ruth,202,3,2.00
998,Junky,371,2,2.00


Получили данные о количестве обзоров и средней оценке для каждой книги.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц (так мы исключим из анализа брошюры)

In [10]:
query = '''
SELECT p.publisher,
       p.publisher_id,
       COUNT(b.book_id) AS count_books
       
FROM publishers AS p
LEFT OUTER JOIN books AS b ON p.publisher_id=b.publisher_id 

WHERE num_pages > 50
GROUP BY p.publisher_id
ORDER BY count_books DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,publisher_id,count_books
0,Penguin Books,212,42


Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books (42 книги).

### Автор с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)

In [18]:
query = '''
SELECT a.author,
       a.author_id,
       AVG(r.rating) AS avg_rating
       
FROM (SELECT book_id
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(rating_id) >= 50) AS pb
INNER JOIN books AS b ON pb.book_id = b.book_id
INNER JOIN authors AS a ON b.author_id = a.author_id
INNER JOIN ratings AS r ON b.book_id = r.book_id

GROUP BY a.author, a.author_id
ORDER BY avg_rating DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author,author_id,avg_rating
0,J.K. Rowling/Mary GrandPré,236,4.287097


Автор с самой высокой средней оценкой книг J.K. Rowling (4,29).

### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''
SELECT AVG(t.review_count) AS avg_review

FROM (SELECT COUNT(review_id) AS review_count
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 48)
      GROUP BY username) AS t;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, 24.

### Общий вывод

В данном проекте выполнен анализ базы данных крупного сервиса для чтения книг по подписке. В ходе анализа установлено:
1. После 1 января 2000 года вышло 819 книг.
2. Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books (42 книги).
3. Автор с самой высокой средней оценкой книг J.K. Rowling (4,29).
4. Пользователи, которые поставили больше 48 оценок,  написали в среднем 24 обзора книг.

Также получены данные о количестве обзоров и средней оценке для каждой книги.